In [1]:
from tensorflow.keras.models import load_model
import numpy as np
import cv2
from PIL import Image
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

# Load your trained model
model = load_model('CNN_model.h5')


ModuleNotFoundError: No module named 'google.colab'

In [3]:
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename


In [4]:
def preprocess_image(image_path):
    img = Image.open(image_path).convert('L')  # Convert to greyscale
    img = img.resize((28, 28), Image.ANTIALIAS)  # Resize to 28x28
    img_array = np.array(img)
    img_array = img_array / 255.0  # Normalize to 0-1
    img_array = img_array.reshape(1, 28, 28, 1)  # Reshape to fit model input
    return img_array


In [12]:
def predict(image_path):
    img_array = preprocess_image(image_path)
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    print(f'Predicted Alphabet: {chr(predicted_class + 65)}')  # Assuming A=0, B=1, ...

# Capture an image from the webcam
image_path = take_photo()

# Make a prediction
predict(image_path)


<IPython.core.display.Javascript object>

1/1 [==============================] - 0s 20ms/step
Predicted Alphabet: P


<ipython-input-4-15a00f29b395>:3: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((28, 28), Image.ANTIALIAS)  # Resize to 28x28
